In [ ]:
!pip install -qU crewai langchain_openai crewai 'crewai[tools]' google-search-results

###Setup API Keys

In [ ]:
import os
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

###Initialize Models

In [ ]:
from langchain_openai import ChatOpenAI
from crewai import LLM
Gemini = LLM(
    model="gemini/gemini-2.0-flash",)



###1. AI Travel Agent 🧳


In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

# Initialize tools
search_tool = SerperDevTool()

def create_travel_agent(destination, preferences):
    # Define the Travel Agent
    travel_agent = Agent(
        role="Travel Planner",
        goal=f"Plan a trip to {destination} based on user preferences",
        backstory="An expert travel assistant trained to create personalized travel itineraries",
        verbose=True,
        tools=[search_tool],
        llm=Gemini
    )

    # Define the Local Expert Agent
    local_expert = Agent(
        role="Local Expert",
        goal=f"Provide insider knowledge and cultural insights about {destination}",
        backstory=f"A longtime resident and cultural expert of {destination} with deep knowledge of local customs, hidden gems, and authentic experiences",
        verbose=True,
        tools=[search_tool],
        llm=Gemini
    )

    # Define the Travel Planning Task
    travel_task = Task(
        description=f"Find the best places to visit, stay, and eat in {destination} based on preferences: {preferences}",
        expected_output=f"A detailed travel itinerary for {destination} including places to visit, accommodations, and restaurants.",
        agent=travel_agent
    )

    # Define the Local Insights Task
    local_task = Task(
        description=f"Provide insider tips, cultural insights, and hidden gems for {destination}. Consider local customs, best times to visit specific locations, and authentic local experiences.",
        expected_output=f"A comprehensive guide of insider knowledge about {destination}, including cultural tips, local customs, and secret spots not found in typical tourist guides.",
        agent=local_expert
    )

    # Create and Run the Crew
    crew = Crew(
        agents=[travel_agent, local_expert],
        tasks=[travel_task, local_task],
        verbose=True,
        process=Process.sequential
    )

    result = crew.kickoff(inputs={"destination": destination, "preferences": preferences})
    return result

### Execute the agent


In [ ]:
destination = input("Enter your travel destination (e.g., Paris, Bali): ")
preferences = input("Enter your preferences (e.g., adventure, luxury, budget-friendly): ")
travel_result = create_travel_agent(destination, preferences)
print("Travel Itinerary:")
print(travel_result)